# Projet .py en streamlit

Bug actuel :  A tester sur plusieurs cas de figure et optimiser le temps de running (dataset 2013 à 2019 , 1,1M de lignes) (TREEEES LENT)

Pour le test , le plus gros échantillon est : Francais naiss/ 60 ou plus / Grande aire urb / Certificat etude primaire / Féminin / Couple sans enfant / CSP non renseignée (39 000 échantillons)

In [ ]:
# Import necessary libraries
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Load the dataset
df_commun = pd.read_csv(r"C:\Users\Alex\df_final.csv")

# Define dictionaries for labels
NFRRED_dict = {
    None: "Non renseigné",
    1: "Français de naissance",
    2: "Français par naturalisation",
    3: "Étranger"
}

AGE5_dict = {
    0: "De 0 à 14 ans",
    15: "De 15 à 29 ans",
    30: "De 30 à 39 ans",
    40: "De 40 à 49 ans",
    50: "De 50 à 59 ans",
    60: "60 ans ou plus"
}

CATAU2010R_dict = {
    1: "Grandes aires urbaines",
    2: "Autres aires",
    3: "Autre commune multipolarisée",
    4: "Commune isolée hors influence des pôles"
}

DIP11_dict = {
    None: "Non renseigné",
    10: "Licence (L3)",
    11: "Écoles niveau licence et au-delà",
    30: "DEUG",
    31: "BTS",
    33: "Paramédical et social (niveau bac+2)",
    41: "Baccalauréat général",
    42: "Baccalauréat technologique",
    50: "CAP",
    60: "Brevet des collèges",
    70: "Certificat d'Études Primaires",
    71: "Sans diplôme"
}

SEXE_dict = {
    1: "Masculin",
    2: "Féminin"
}

TYPMEN7_dict = {
    1: "Ménage d'une seule personne",
    2: "Famille monoparentale",
    3: "Couple sans enfant",
    4: "Couple avec enfant(s)",
    5: "Ménage de plusieurs personnes ayant toutes un lien de parenté avec la personne de référence du ménage",
    6: "Ménage de plusieurs personnes n'ayant pas toutes un lien de parenté avec la personne de référence du ménage",
    9: "Autres ménages complexes de plus d'un personne"
}

CSTOT_dict = {
    None: "Sans objet (ACTEU non renseigné)",
    0: "Non renseigné (pour les actifs)",
    10: "Agriculteurs exploitants",
    11: "Agriculteurs sur petite exploitation",
    12: "Agriculteurs sur moyenne exploitation",
    13: "Agriculteurs sur grande exploitation",
    21: "Artisans",
    22: "Commerçants et assimilés",
    23: "Chefs d'entreprise de 10 salariés ou plus",
    31: "Professions libérales",
    33: "Cadres de la fonction publique",
    34: "Professeurs",
    35: "Professions de l'information",
    37: "Cadres administratifs et commerciaux d'entreprise",
    38: "Ingénieurs et cadres techniques d'entreprise",
    42: "Professeurs des écoles",
    43: "Professions intermédiaires de la santé et du travail social",
    44: "Clergé",
    45: "Professions intermédiaires administratives de la fonction publique",
    46: "Professions intermédiaires administratives et commerciales des entreprises",
    47: "Techniciens",
    48: "Contremaîtres",
    52: "Employés civils et agents de service de la fonction publique",
    53: "Policiers et militaires",
    54: "Employés administratifs d'entreprise",
    55: "Employés de commerce",
    56: "Personnels des services directs aux particuliers",
    62: "Ouvriers qualifiés de type industriel",
    63: "Ouvriers qualifiés de type artisanal",
    64: "Chauffeurs",
    65: "Ouvriers qualifiés de la manutention",
    67: "Ouvriers non qualifiés de type industriel",
    68: "Ouvriers non qualifiés de type artisanal",
    69: "Ouvriers agricoles",
    71: "Anciens agriculteurs exploitants",
    72: "Anciens artisans",
    74: "Anciens cadres",
    75: "Anciennes professions intermédiaires",
    77: "Anciens employés",
    78: "Anciens ouvriers",
    81: "Chômeurs n'ayant jamais travaillé",
    83: "Militaires du contingent",
    84: "Elèves",
    85: "Personnes de moins de 60 ans n'ayant jamais travaillé",
    86: "Personnes de 60 ans ou plus n'ayant jamais travaillé"
}

# Define dictionary for ANCINACT labels
ANCINACT_dict = {
    0: "Sans objet (actifs occupés d'une part chômeurs ou inactifs n'ayant jamais travaillé d'autre part) ou non renseigné",
    1: "Moins d'un mois",
    2: "De 1 mois à moins de 3 mois",
    3: "De 3 mois à moins de 6 mois",
    4: "De 6 mois à moins d'un an",
    5: "De 1 an à moins d'un an et demi",
    6: "De 1 an et demi à moins de 2 ans",
    7: "De 2 ans à moins de 3 ans",
    8: "3 ans ou plus"
}

# Dictionary for labels
label_dicts = {
    'NFRRED': NFRRED_dict,
    'AGE5': AGE5_dict,
    'CATAU2010R': CATAU2010R_dict,
    'DIP11': DIP11_dict,
    'SEXE': SEXE_dict,
    'TYPMEN7': TYPMEN7_dict,
    'CSTOT': CSTOT_dict
}

# Create dropdown menus for factors
with st.form(key='my_form'):
    nat = st.selectbox("Choisissez votre nationalité", list(NFRRED_dict.values()))
    age = st.selectbox("Choisissez votre âge", list(AGE5_dict.values()))
    region = st.selectbox("Choisissez votre région", list(CATAU2010R_dict.values()))
    diplome = st.selectbox("Choisissez votre niveau de diplôme", list(DIP11_dict.values()))
    sexe = st.selectbox("Choisissez votre genre", list(SEXE_dict.values()))
    menage = st.selectbox("Choisissez votre type de ménage", list(TYPMEN7_dict.values()))
    csp = st.selectbox("Choisissez votre CSP", list(CSTOT_dict.values()))
    
    submit_button = st.form_submit_button(label='Prédire')

# Make prediction if the button is clicked
if submit_button:
    # Prepare input and output data
    X = df_commun[['NFRRED', 'AGE5', 'CATAU2010R', 'DIP11', 'SEXE', 'TYPMEN7', 'CSTOT']]
    y = df_commun["ANCINACT"]
    
    # Impute missing values with median
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X)
    

    
    # Convert categorical variables to numerical using one-hot encoding
    input_data = np.array([list(NFRRED_dict.keys())[list(NFRRED_dict.values()).index(nat)],
                           list(AGE5_dict.keys())[list(AGE5_dict.values()).index(age)],
                           list(CATAU2010R_dict.keys())[list(CATAU2010R_dict.values()).index(region)],
                           list(DIP11_dict.keys())[list(DIP11_dict.values()).index(diplome)],
                           list(SEXE_dict.keys())[list(SEXE_dict.values()).index(sexe)],
                           list(TYPMEN7_dict.keys())[list(TYPMEN7_dict.values()).index(menage)],
                           list(CSTOT_dict.keys())[list(CSTOT_dict.values()).index(csp)]]).reshape(1, -1)
    
    # Ensure that both training and input data have the same columns after one-hot encoding
    X_encoded = pd.get_dummies(X, columns=['NFRRED', 'AGE5', 'CATAU2010R', 'DIP11', 'SEXE', 'TYPMEN7', 'CSTOT'])
    
    # Ensure the input_data shape matches the X_encoded shape
    input_data_encoded = pd.get_dummies(pd.DataFrame(input_data, columns=X.columns), columns=X.columns)
    
    # Align the input_data_encoded with X_encoded columns
    input_data_encoded = input_data_encoded.reindex(columns=X_encoded.columns, fill_value=0)
    
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    
    # Train RandomForestClassifier model with imputed data
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict using the encoded input data
    prediction = model.predict(input_data_encoded)
    
    # Map prediction to its corresponding label
    predicted_label = ANCINACT_dict[prediction[0]]
    
    # Display the prediction
    st.write("Votre temps de chômage, à partir des paramètres choisis, est de :", predicted_label)

## V2 avec possibilité de comparatif (en chantier) :

Bug actuel : ValueError: Length of values (186) does not match length of index (1179770) (logique...)

In [ ]:
# Import des bibliothèques nécessaires
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


# Charger le jeu de données
df_commun = pd.read_csv(r"C:\Users\Alex\df_final.csv")

# Définition des dictionnaires pour les labels
NFRRED_dict = {
    1: "Français de naissance",
    2: "Français par naturalisation",
    3: "Étranger"
}

AGE5_dict = {
    0: "De 0 à 14 ans",
    15: "De 15 à 29 ans",
    30: "De 30 à 39 ans",
    40: "De 40 à 49 ans",
    50: "De 50 à 59 ans",
    60: "60 ans ou plus"
}

CATAU2010R_dict = {
    1: "Grandes aires urbaines",
    2: "Autres aires",
    3: "Autre commune multipolarisée",
    4: "Commune isolée hors influence des pôles"
}

DIP11_dict = {
    10: "Licence (L3)",
    11: "Écoles niveau licence et au-delà",
    30: "DEUG",
    31: "BTS",
    33: "Paramédical et social (niveau bac+2)",
    41: "Baccalauréat général",
    42: "Baccalauréat technologique",
    50: "CAP",
    60: "Brevet des collèges",
    70: "Certificat d'Études Primaires",
    71: "Sans diplôme"
}

SEXE_dict = {
    1: "Masculin",
    2: "Féminin"
}

TYPMEN7_dict = {
    1: "Ménage d'une seule personne",
    2: "Famille monoparentale",
    3: "Couple sans enfant",
    4: "Couple avec enfant(s)",
    5: "Ménage de plusieurs personnes ayant toutes un lien de parenté avec la personne de référence du ménage",
    6: "Ménage de plusieurs personnes n'ayant pas toutes un lien de parenté avec la personne de référence du ménage",
    9: "Autres ménages complexes de plus d'un personne"
}

CSER_dict = {
    0: "Non renseigné",
    1: "Agriculteurs exploitants",
    2: "Artisans",
    3: "Cadres et professions intellectuelles supérieures",
    4: "Professions intermédiaires",
    5: "Employés",
    6: "Ouvriers",
    8: "Chômeurs n'ayant jamais travaillé",
}

# Créer les menus déroulants pour les facteurs
with st.form(key='my_form'):
    nat = st.selectbox("Choisissez votre nationalité", list(NFRRED_dict.values()))
    age = st.selectbox("Choisissez votre âge", list(AGE5_dict.values()))
    region = st.selectbox("Choisissez votre région", list(CATAU2010R_dict.values()))
    diplome = st.selectbox("Choisissez votre niveau de diplôme", list(DIP11_dict.values()))
    sexe = st.selectbox("Choisissez votre genre", list(SEXE_dict.values()))
    menage = st.selectbox("Choisissez votre type de ménage", list(TYPMEN7_dict.values()))
    csp = st.selectbox("Choisissez votre CSP", list(CSER_dict.values()))
    
    submit_button = st.form_submit_button(label='Prédire')

# Faire une prédiction si le bouton est cliqué
if submit_button:
    # Filtrer les données pour n'inclure que les échantillons correspondant aux paramètres choisis
    filtered_df = df_commun[(df_commun['NFRRED'] == list(NFRRED_dict.keys())[list(NFRRED_dict.values()).index(nat)]) & 
                            (df_commun['AGE5'] == list(AGE5_dict.keys())[list(AGE5_dict.values()).index(age)]) & 
                            (df_commun['CATAU2010R'] == list(CATAU2010R_dict.keys())[list(CATAU2010R_dict.values()).index(region)]) & 
                            (df_commun['DIP11'] == list(DIP11_dict.keys())[list(DIP11_dict.values()).index(diplome)]) & 
                            (df_commun['SEXE'] == list(SEXE_dict.keys())[list(SEXE_dict.values()).index(sexe)]) & 
                            (df_commun['TYPMEN7'] == list(TYPMEN7_dict.keys())[list(TYPMEN7_dict.values()).index(menage)]) & 
                            (df_commun['CSER'] == list(CSER_dict.keys())[list(CSER_dict.values()).index(csp)])]

    st.write("Number of matching samples:", len(filtered_df))

    if len(filtered_df) == 0:
        st.error("Aucun échantillon ne correspond aux paramètres choisis. Veuillez réessayer avec d'autres valeurs.")
    else:
        # Prétraitement des données
        X = filtered_df[['NFRRED', 'AGE5', 'CATAU2010R', 'DIP11', 'SEXE', 'TYPMEN7', 'CSER']]
        X_encoded = pd.get_dummies(X, columns=['NFRRED', 'AGE5', 'CATAU2010R', 'DIP11', 'SEXE', 'TYPMEN7', 'CSER'])

        # Fractionner les données
        y = filtered_df["ANCINACT"]
        X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

        # Entraîner le modèle RandomForestClassifier
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # Préparer les données d'entrée
        input_data = np.array([nat, age, region, diplome, sexe, menage, csp]).reshape(1, -1)
        input_data_encoded = pd.get_dummies(pd.DataFrame(input_data, columns=X.columns), columns=X.columns)
        input_data_encoded = input_data_encoded.reindex(columns=X_encoded.columns, fill_value=0)

        # Prédire en utilisant les données d'entrée encodées
        prediction = model.predict(input_data_encoded)

        # Dictionnaire pour les labels
        ANCINACT_dict = {
            0: "Sans objet (actifs occupés d'une part chômeurs ou inactifs n'ayant jamais travaillé d'autre part) ou non renseigné",
            1: "Moins d'un mois",
            2: "De 1 mois à moins de 3 mois",
            3: "De 3 mois à moins de 6 mois",
            4: "De 6 mois à moins d'un an",
            5: "De 1 an à moins d'un an et demi",
            6: "De 1 an et demi à moins de 2 ans",
            7: "De 2 ans à moins de 3 ans",
            8: "3 ans ou plus"
        }

        # Afficher la prédiction
        st.write("Votre temps de chômage, à partir des paramètres choisis, est de :", ANCINACT_dict[prediction[0]])

    # Données pour la première source
    x1 = df_commun["SEXE"]
    y1 = df_commun['ANCINACT']

    # Données pour la deuxième source (prédiction)
    # Utilisons les valeurs prédites pour toutes les lignes du jeu de données
    predictions = model.predict(X_encoded)
    df_commun['Prediction'] = predictions
    x2 = df_commun["SEXE"]
    y2 = df_commun['Prediction'].value()

    # Création du graphique
    plt.bar(x1, y1, width=0.4, align='center', label='Source 1')
    plt.bar(x2, y2, width=0.4, align='center', label='Prédiction')

    # Étiquettes et légendes
    plt.xlabel('SEXE')
    plt.ylabel('ANCINAT / Prédiction')
    plt.title('Comparaison entre ANCINAT et Prédiction')
    plt.legend()

    # Affichage du graphique
    st.pyplot(plt)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=235804c3-56ea-488a-867b-f3378205252b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>